# Layered Queueing Network with Cache - Three Hosts

This example demonstrates a layered queueing network (LQN) with cache and backend service distributed across three hosts. The model includes cache hits that are served locally and cache misses that require backend service calls.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.lang.layered.*
import jline.solvers.ln.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.util.matrix.*

In [ ]:
val model = LayeredNetwork("LQNwithCaching")
val nUsers = 1
val nTokens = 1

// Client processor and task (Host 1)
val P1 = Processor(model, "P1", 1, SchedStrategy.PS)
val T1 = Task(model, "T1", nUsers, SchedStrategy.REF).on(P1)
val E1 = Entry(model, "E1").on(T1)

println("Client Configuration (Host 1):")
println("- Processor P1: 1 server, PS scheduling")
println("- Task T1: $nUsers users, REF scheduling")
println("- Entry E1: Client entry point")

In [ ]:
// Cache task configuration (Host 2)
val totalItems = 4
val cacheCapacity = 2  // Single cache level with 2 items

// Create uniform access probabilities
val accessProbs = Matrix(1, totalItems).fill(1.0 / totalItems)
val pAccess = DiscreteSampler(accessProbs)

val PC = Processor(model, "Pc", 1, SchedStrategy.PS)
val C2 = CacheTask(model, "CT", totalItems, cacheCapacity, ReplacementStrategy.RR, nTokens).on(PC)
val I2 = ItemEntry(model, "IE", totalItems, pAccess).on(C2)

println("\nCache Configuration (Host 2):")
println("- Total items: $totalItems")
println("- Cache capacity: $cacheCapacity")
println("- Replacement strategy: Round Robin (RR)")
println("- Access pattern: Uniform (${1.0/totalItems} probability per item)")
println("- Processor Pc: 1 server, PS scheduling")
println("- Cache tokens: $nTokens")

In [ ]:
// Backend service configuration (Host 3)
val P3 = Processor(model, "P2", 1, SchedStrategy.PS)
val T3 = Task(model, "T2", 1, SchedStrategy.FCFS).on(P3)
val E3 = Entry(model, "E2").on(T3)
val A3 = Activity(model, "A2", Exp(5.0)).on(T3).boundTo(E3).repliesTo(E3)

println("\nBackend Service Configuration (Host 3):")
println("- Processor P2: 1 server, PS scheduling")
println("- Task T2: 1 task, FCFS scheduling")
println("- Entry E2: Backend service entry")
println("- Activity A2: Backend processing (Exp(5.0))")

In [ ]:
// Define activities and cache logic
val A1 = Activity(model, "A1", Immediate()).on(T1).boundTo(E1).synchCall(I2, 1)
val AC2 = Activity(model, "Ac", Immediate()).on(C2).boundTo(I2)
val AC2h = Activity(model, "Ac_hit", Exp(1.0)).on(C2).repliesTo(I2)  // Cache hit - local service
val AC2m = Activity(model, "Ac_miss", Exp(0.5)).on(C2).synchCall(E3, 1).repliesTo(I2)  // Cache miss - backend call

// Add cache access precedence
C2.addPrecedence(ActivityPrecedence.CacheAccess(AC2, arrayOf(AC2h, AC2m)))

println("\nActivity Configuration:")
println("- A1: Client activity (immediate) calls cache")
println("- Ac: Cache access activity (immediate)")
println("- Ac_hit: Cache hit activity (Exp(1.0) - fast local service)")
println("- Ac_miss: Cache miss activity (Exp(0.5) + backend call)")
println("- Cache access precedence: Ac → {Ac_hit, Ac_miss}")
println("- Backend call: Ac_miss → E2 (Host 3)")

In [ ]:
// Solve with Layered Network solver using NC
try {
    val lnOptions = LN.defaultOptions()
    lnOptions.verbose = true
    
    val ncOptions = NC.defaultOptions()
    ncOptions.verbose = false
    
    val solver1 = LN(model) { subModel -> NC(subModel, ncOptions) }
    solver1.options = lnOptions
    
    val avgTable1 = solver1.avgTable
    println("\n=== Layered Network Results (NC Solver) ===")
    avgTable1.print()
    
} catch (e: Exception) {
    println("\nLN with NC failed: ${e.message}")
}

In [ ]:
// Solve with Layered Network solver using MVA for comparison
try {
    val lnOptions = LN.defaultOptions()
    lnOptions.verbose = true
    
    val mvaOptions = MVA.defaultOptions()
    mvaOptions.verbose = false
    
    val solver2 = LN(model) { subModel -> MVA(subModel, mvaOptions) }
    solver2.options = lnOptions
    
    val avgTable2 = solver2.avgTable
    println("\n=== Layered Network Results (MVA Solver) ===")
    avgTable2.print()
    
} catch (e: Exception) {
    println("\nLN with MVA failed: ${e.message}")
}

In [ ]:
// Analyze the three-host distributed cache system
println("\n=== Three-Host Distributed Cache Analysis ===")
println("This model demonstrates a distributed architecture:")
println("\n1. Three-Tier Architecture:")
println("   - Host 1: Client tier (users generate requests)")
println("   - Host 2: Cache tier (intermediate caching layer)")
println("   - Host 3: Backend tier (authoritative data source)")
println("\n2. Cache Behavior:")
println("   - Cache hits: Served locally on Host 2 (fast)")
println("   - Cache misses: Require call to Host 3 (slower)")
println("   - Hit ratio ≈ $cacheCapacity/$totalItems = ${cacheCapacity.toDouble()/totalItems}")
println("\n3. Performance Characteristics:")
println("   - Hit latency: Exp(1.0) local processing")
println("   - Miss latency: Exp(0.5) cache processing + Exp(5.0) backend")
println("   - Backend is significantly slower (5.0 vs 1.0 rate)")
println("   - Network delays between hosts (implicit in LQN)")
println("\n4. Traffic Flow:")
println("   - Client → Cache (always)")
println("   - Cache → Backend (only on misses)")
println("   - Response path: Backend → Cache → Client")
println("\n5. Scalability Implications:")
println("   - Cache reduces backend load by factor of hit ratio")
println("   - Backend load = arrival rate × miss ratio")
println("   - Cache effectiveness critical for system performance")
println("\nReal-world Applications:")
println("- Web application with Redis cache and database")
println("- CDN with origin server")
println("- Microservices with caching layer")
println("- Distributed database with cache nodes")
println("\nPerformance Tuning Insights:")
val hitRatio = cacheCapacity.toDouble() / totalItems
val missRatio = 1.0 - hitRatio
println("- Expected backend load reduction: ${(hitRatio * 100).toInt()}%")
println("- Cache miss penalty: ~6x slower (0.5 + 5.0 vs 1.0)")
println("- Increasing cache capacity from $cacheCapacity to ${cacheCapacity+1} would improve hit ratio to ${(cacheCapacity+1).toDouble()/totalItems}")

This example demonstrates:

1. **Distributed Layered Architecture**: 
   - Three-tier system with client, cache, and backend layers
   - Inter-host communication via synchronous calls
   - Network effects implicit in LQN modeling
2. **Cache-Backend Integration**:
   - Cache hits served locally (fast response)
   - Cache misses trigger backend calls (slower response)
   - Cache acts as performance multiplier for backend
3. **Performance Differentiation**:
   - Local cache processing: Exp(1.0)
   - Cache miss processing: Exp(0.5) + backend call
   - Backend processing: Exp(5.0) (much slower)
4. **Solver Comparison**:
   - NC (Normalizing Constant) vs MVA solvers
   - Different numerical approaches for same model
   - Validation of solution consistency
5. **Scalability Analysis**:
   - Cache reduces backend load proportional to hit ratio
   - Backend becomes bottleneck when cache ineffective
   - System performance highly sensitive to cache sizing

This model is essential for:
- Distributed system design
- Cache sizing decisions
- Backend capacity planning
- Understanding cache effectiveness in reducing system load
- Analyzing multi-tier application performance